In [ ]:
I_GPU = 0

In [ ]:
# %load_ext autoreload
# %autoreload 2
import os
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap

DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)

from torch_points3d.datasets.segmentation.multimodal.s3dis_area1_office1 import S3DISFusedDataset
from torch_points3d.datasets.segmentation.multimodal import IGNORE_LABEL
from torch_points3d.visualization.multimodal_data import visualize_mm_data

# import plotly.offline as pyo
# pyo.init_notebook_mode()
import plotly.io as pio
# pio.renderers
pio.renderers.default = 'jupyterlab'  # for local jupyterlab

# Ror remote HPC jupyterhub.
# Other working (but seemingly slower) options are: 'sphinx_gallery' and 'iframe'
# pio.renderers.default = 'iframe_connected' 

_ = torch.cuda.is_available()
_ = torch.cuda.memory_allocated()
torch.cuda.set_device(I_GPU)

## Helpers

In [ ]:
from torch_geometric.data import Data, Batch
from torch import Tensor

OBJECT_COLOR = [
        [233, 229, 107],  #'ceiling' .-> .yellow
        [95, 156, 196],  #'floor' .-> . blue
        [179, 116, 81],  #'wall'  ->  brown
        [241, 149, 131],  #'beam'  ->  salmon
        [81, 163, 148],  #'column'  ->  bluegreen
        [77, 174, 84],  #'window'  ->  bright green
        [108, 135, 75],  #'door'   ->  dark green
        [41, 49, 101],  #'chair'  ->  darkblue
        [79, 79, 76],  #'table'  ->  dark grey
        [223, 52, 52],  #'bookcase'  ->  red
        [89, 47, 95],  #'sofa'  ->  purple
        [81, 109, 114],  #'board'   ->  grey
        [233, 233, 229],  #'clutter'  ->  light grey
        [0, 0, 0],  # unlabelled .->. black
    ]

CLASSES = [
    'ceiling',
    'floor',
    'wall',
    'beam',
    'column',
    'window',
    'door',
    'chair',
    'table',
    'bookcase',
    'sofa',
    'board',
    'clutter',
    'unlabelled',
]

In [ ]:
from torch_geometric.transforms import *
from torch_points3d.core.data_transform import *
from torch_points3d.core.data_transform.multimodal.image import *
from torch_points3d.datasets.base_dataset import BaseDataset
from torch_points3d.datasets.base_dataset_multimodal import BaseDatasetMM

def sample_real_data(tg_dataset, idx=0):
    """
    Temporarily remove the 3D and 2D transforms affecting the point 
    positions and images from the dataset to better visualize points 
    and images relative positions.
    """
    transform = tg_dataset.transform
    tg_dataset.transform = BaseDataset.remove_transform(transform, [Center, RandomNoise,
        RandomRotate, RandomScaleAnisotropic, RandomSymmetry, DropFeature, AddFeatsByKeys])
    
    transform_image = tg_dataset.transform_image
#     tg_dataset.transform_image = BaseDatasetMM.remove_multimodal_transform(transform_image, [ToFloatImage, Normalize, AddPixelHeightFeature,])
#     tg_dataset.transform_image = BaseDatasetMM.remove_multimodal_transform(transform_image, [ToFloatImage, Normalize, PickImagesFromMappingArea, AddPixelHeightFeature, ])
#     tg_dataset.transform_image = BaseDatasetMM.remove_multimodal_transform(transform_image, [ToFloatImage, Normalize, PickImagesFromMappingArea, AddPixelHeightFeature,PickImagesFromMemoryCredit ])
    tg_dataset.transform_image = BaseDatasetMM.remove_multimodal_transform(transform_image, [ToFloatImage, Normalize, AddPixelHeightFeature,PickImagesFromMemoryCredit ])

    out = tg_dataset[idx]
    
    tg_dataset.transform = transform
    tg_dataset.transform_image = transform_image
    return out

# Load config

In [ ]:
from omegaconf import OmegaConf
from torch_points3d.utils.config import hydra_read

# Set root to the DATA drive, where the data was downloaded
# DATA_ROOT = '/mnt/fa444ffd-fdb4-4701-88e7-f00297a8e29b/projects/datasets/s3dis'  # ???
# DATA_ROOT = '/media/drobert-admin/DATA/datasets/s3dis'  # IGN DATA
DATA_ROOT = '/media/drobert-admin/DATA2/datasets/s3dis'  # IGN DATA2
# DATA_ROOT = '/var/data/drobert/datasets/s3dis'  # AI4GEO
# DATA_ROOT = '/home/qt/robertda/scratch/datasets/s3dis'  # CNES
# DATA_ROOT = '/raid/datasets/pointcloud/data/s3dis'  # ENGIE

overrides = [
    'task=segmentation',
    'data=segmentation/multimodal/s3disfused/area1-office1_no3d_5cm_768x384-exact_no-pixel-height',
    'models=segmentation/multimodal/no3d',
    'model_name=RGB_ResNet18PPM_mean-feat',
    'data.fold=1',
    'data.sample_per_epoch=8',
    f"data.dataroot={os.path.join(DATA_ROOT, '5cm_exact_768x384')}",
]

cfg = hydra_read(overrides)

# print(OmegaConf.to_yaml(cfg))

## Load S3DIS dataset

In [ ]:
from time import time
start = time()
    
dataset = S3DISFusedDataset(cfg.data)
# print(dataset)

print(f"Time = {time() - start:0.1f} sec.")

In [ ]:
# Get multimodal spherical sample
# mm_data = sample_real_data(dataset.test_dataset[0], idx=100)
# mm_data = sample_real_data(dataset.train_dataset)
# mm_data = sample_real_data(dataset.val_dataset)
mm_data = sample_real_data(dataset.test_dataset[0], idx=1)
print(mm_data)

credit_used = sum([np.prod(im.img_size) * im.num_views for im in mm_data.modalities['image']])
credit_max = dataset.train_transform_image.transforms[-1].credit
print(f"Used pixel credit: {credit_used} / {credit_max} = {credit_used / credit_max * 100:0.1f} %")

In [ ]:
visualize_mm_data(mm_data, class_names=CLASSES, class_colors=OBJECT_COLOR, figsize=1000, voxel=0.05, show_3d=True, show_2d=True, color_mode='y', alpha=5, pointsize=5)